In [63]:
#IMPORTS
import sys
from collections import defaultdict
from itertools import combinations
import random
import numpy as np
import pdb

from pyspark import SparkContext
sc=SparkContext.getOrCreate()

#FUNCTIONS

def parseVectorOnUser(line):
    '''
    Parse each line of the specified data file, assuming a "," delimiter.
    Key is user_id, converts each rating to a float.
    '''
    line = line.split(",")
    return int(line[0]),(int(line[1]),float(line[2]))

def parseVectorOnItem(line):
    '''
    Parse each line of the specified data file, assuming a "," delimiter.
    Key is item_id, converts each rating to a float.
    '''
    line = line.split(",")
    return line[1],(line[0],float(line[2]))

def sampleInteractions(item_id,users_with_rating,n):
    '''
    For items with # interactions > n, replace their interaction history
    with a sample of n users_with_rating
    '''
    if len(users_with_rating) > n:
        return item_id, random.sample(users_with_rating,n)
    else:
        return item_id, users_with_rating

def findUserPairs(item_id,users_with_rating):
    '''
    For each item, find all user-user pairs combos. (i.e. users with the same item) 
    '''
    for user1,user2 in combinations(users_with_rating,2):
        return (user1[0],user2[0]),(user1[1],user2[1])


In [64]:
'''
trainSet = sc.textFile("../train.csv").filter(lambda x: "userId" not in x)
trainSet = trainSet.map(parseVectorOnUser)
trainSet.take(5)
'''
testSet=sc.parallelize((0,(2,1.0)),(0,(0,4.0)),(1,(0,1.0)),(1,(1,3.0)),(1,(2,3.0)),(1,(3,4.0)),(2,(0,2.0)),(2,(1,4.0)),(3,(1,5.0)),(3,(3,8.0)))

SyntaxError: invalid syntax (<ipython-input-64-cd54d7282678>, line 6)

In [65]:
print("All the test set")
testSet.take(5)

All the test set


[(0, (2, 1.0)), (0, (0, 4.0)), (1, (0, 1.0)), (1, (1, 3.0)), (1, (2, 3.0))]

In [66]:
print("Testset grouped by key")
testSetToPrint=testSet.groupByKey()\
                .map(lambda x: (x[0],list(x[1])))
print(testSetToPrint.collect())
item_user_pairs=testSet.groupByKey().cache()

Testset grouped by key
[(0, [(2, 1.0), (0, 4.0)]), (1, [(0, 1.0), (1, 3.0), (2, 3.0), (3, 4.0)]), (2, [(0, 2.0), (1, 4.0)]), (3, [(1, 5.0), (3, 8.0)])]


In [67]:
#only consider item with more than 1 interaciton
pairwise_users = item_user_pairs.filter(
        lambda p: len(p[1]) > 1).map(
        lambda p: findUserPairs(p[0],p[1])) #p[0] is the item, p[1] is the list of users 

pairwise_users.collect()

[None, None, None, None]

In [52]:
c=combinations([(0, 1.0), (1, 3.0), (2, 3.0), (3, 4.0)],2)
for el in c:
    print(el)

((0, 1.0), (1, 3.0))
((0, 1.0), (2, 3.0))
((0, 1.0), (3, 4.0))
((1, 3.0), (2, 3.0))
((1, 3.0), (3, 4.0))
((2, 3.0), (3, 4.0))
